Dans ce TP, nous allons entraîner et tester un RNN pour la génération de texte. Plus précisément, le modèle que nous allons construire sera capable, étant donnée une séquence de caractères, de prédire le prochain caractère le plus probable.

![model](https://drive.google.com/uc?id=1syE1phix6Pu-b8y9ktol0thCdC2lzmlV
)

Il sera alors possible, partant d'une chaîne de caractères, de réaliser plusieurs inférences du modèle successivement pour générer la suite de la phrase.

![inference](https://drive.google.com/uc?id=1T6J3UgFV4Q2JhJm3984HhJIkH7ukWkb7
)

Les phrases suivantes ont été obtenues à l'issue d'un entraînement du modèle sur une base de données regroupant les tweets de Donald Trump (à partir respectivement des débuts de phrase 'China', 'Obama', et 'Mo'). Même si les phrases ne sont pas complètement correctes, le modèle arrive à générer des mots existants (pour la plupart) et à les enchaîner d'une manière tout de même relativement crédible !



<pre>
China on dollars are sources to other things!The Fake News State approvement is smart, restlected & unfair

Obama BEAT!Not too late. This is the only requirement. Also, the Fake News is running a big democrats want

More system. See you really weak!Thank you. You and others just doesn’t exist.
</pre>

L'objectif du TP est de découvrir comment préparer la base de données, implémenter et entraîner le modèle, et réaliser l'inférence. Seules quelques lignes sont à compléter.

A vous de vous emparer du code ci-dessous pour essayer d'améliorer les performances du modèle et de générer les phrases les plus crédibles possibles !

# Téléchargement des données

In [41]:
import tensorflow as tf

import numpy as np
import os
import time

Téléchargement des données

In [42]:
path_to_file = tf.keras.utils.get_file('realdonaltrump.csv', 'https://drive.google.com/uc?export=download&id=1s1isv9TQjGiEr2gG__8bOdBFvQlmepRt')

On commence par extraire les tweets du CSV (notez qu'il y a d'autres métadonnées dans le fichier, comme le nombre de retweets par exemple, qui pourraient être utilisées pour d'autres tâches).

In [43]:
import csv
tweets = []
text = ''
with open(path_to_file, newline='') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        tweets.append(row['content'])
        text += row['content']

# Affichage des 10 premiers tweets
print(tweets[:10])

['Be sure to tune in and watch Donald Trump on Late Night with David Letterman as he presents the Top Ten List tonight!', 'Donald Trump will be appearing on The View tomorrow morning to discuss Celebrity Apprentice and his new book Think Like A Champion!', 'Donald Trump reads Top Ten Financial Tips on Late Show with David Letterman: http://tinyurl.com/ooafwn - Very funny!', 'New Blog Post: Celebrity Apprentice Finale and Lessons Learned Along the Way: http://tinyurl.com/qlux5e', '"My persona will never be that of a wallflower - I’d rather build walls than cling to them" --Donald J. Trump', 'Miss USA Tara Conner will not be fired - "I\'ve always been a believer in second chances." says Donald Trump', 'Listen to an interview with Donald Trump discussing his new book, Think Like A Champion: http://tinyurl.com/qs24vl', '"Strive for wholeness and keep your sense of wonder intact." --Donald J. Trump http://tinyurl.com/pqpfvm', 'Enter the "Think Like A Champion" signed book and keychain conte

In [44]:
# Nombre total de caractères du dataset
print(f'Longueur totale du texte: {len(text)} caractères')

Longueur totale du texte: 5701912 caractères


In [45]:
# Extraction des caractères uniques du texte
vocab = sorted(set(text))
print(f'{len(vocab)} unique caractères')

259 unique caractères


In [46]:
# Affichage du vocabulaire
print(vocab)

[' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', '`', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '{', '|', '}', '~', '\x92', '£', '«', '®', '´', 'º', '»', '½', 'É', 'á', 'â', 'è', 'é', 'í', 'ï', 'ñ', 'ò', 'ó', 'ô', 'ö', 'ø', 'ú', 'ğ', 'ı', 'ĺ', 'ō', 'א', 'ב', 'ג', 'ד', 'ה', 'ו', 'ז', 'ח', 'ט', 'י', 'ך', 'כ', 'ל', 'ם', 'מ', 'ן', 'נ', 'ס', 'ע', 'צ', 'ק', 'ר', 'ש', 'ת', '،', 'ء', 'آ', 'أ', 'ؤ', 'ا', 'ب', 'ة', 'ت', 'ج', 'ح', 'خ', 'د', 'ذ', 'ر', 'ز', 'س', 'ش', 'ص', 'ض', 'ط', 'ظ', 'ع', 'ف', 'ق', 'ك', 'ل', 'م', 'ن', 'ه', 'و', 'ي', 'ً', 'چ', 'ژ', 'ک', 'گ', 'ی', '۰', '۴', 'ँ', 'ं', 'अ', 'आ', 'इ', 'उ', 'ए', 'औ', 'क', 'ख', 'ग', 'घ', 'च', 'छ', 'ज', 

# Préparation des données

Il est nécessaire de convertir les caractères dans une représentation admissible par le modèle.

La fonction `tf.keras.layers.StringLookup` convertit les chaînes de caractères en nombre, en reprenant l'indice de chaque caractère dans le vocabulaire établi précédemment.

Il faut cependant commencer par séparer le texte en caractères, comme présenté sur l'exemple ci-dessous.

In [47]:
example_texts = ['abcdefg', 'xyz']

chars = tf.strings.unicode_split(example_texts, input_encoding='UTF-8')
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

On peut ensuire appliquer la fonction `tf.keras.layers.StringLookup` :

In [48]:
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None)

In [49]:
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[65, 66, 67, 68, 69, 70, 71], [88, 89, 90]]>

Pour retrouver un texte à partir de sa représentation numérique (ce sera utile lors de l'étape finale de génération) il faut être capable d'inverser le processus, ce que l'on peut faire avec `tf.keras.layers.StringLookup(..., invert=True)`.  

In [50]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None)

In [51]:
chars = chars_from_ids(ids)
chars

<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>

Enfin, on peut recréer une chaîne de caractères :

In [52]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [53]:
def text_from_ids(ids):
  return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

Il faut maintenant créer les exemples d'apprentissage, ainsi que leurs labels associés. Pour cela, nous allons diviser le texte en séquences, chacune composée de `seq_length` caractères.

Pour chaque séquence constituant un ensemble d'apprentissage, le label à prédire correspondant est une séquence de même longueur dont tous les caractères ont été décalés d'un cran.

Une manière simple de constituer notre base est donc de diviser le texte en séquences de longueur `seq_length+1`, et d'utiliser les `seq_length` premiers caractères comme donnée, et les `seq_length` derniers caractères comme label.


N.B. Cette manière de faire n'est clairement pas optimale ! Certaines séquences vont recouvrir deux tweets successifs, qui n'auront potentiellement aucun lien entre eux !

In [54]:
all_ids = ids_from_chars(tf.strings.unicode_split(text, 'UTF-8'))
all_ids

<tf.Tensor: shape=(5701912,), dtype=int64, numpy=array([35, 69,  1, ..., 56, 21, 18])>

In [55]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)

In [56]:
for ids in ids_dataset.take(10):
    print(chars_from_ids(ids).numpy().decode('utf-8'))

B
e
 
s
u
r
e
 
t
o


In [57]:
seq_length = 50

La méthode `batch` permet de regrouper les caractères du texte en séquences de la longueur voulue.

In [58]:
sequences = ids_dataset.batch(seq_length+1, drop_remainder=True)

for seq in sequences.take(1):
  print(chars_from_ids(seq))

tf.Tensor(
[b'B' b'e' b' ' b's' b'u' b'r' b'e' b' ' b't' b'o' b' ' b't' b'u' b'n'
 b'e' b' ' b'i' b'n' b' ' b'a' b'n' b'd' b' ' b'w' b'a' b't' b'c' b'h'
 b' ' b'D' b'o' b'n' b'a' b'l' b'd' b' ' b'T' b'r' b'u' b'm' b'p' b' '
 b'o' b'n' b' ' b'L' b'a' b't' b'e' b' ' b'N'], shape=(51,), dtype=string)


Voici par exemple les premières séquences extraites du dataset :

In [59]:
for seq in sequences.take(5):
  print(text_from_ids(seq).numpy())

b'Be sure to tune in and watch Donald Trump on Late N'
b'ight with David Letterman as he presents the Top Te'
b'n List tonight!Donald Trump will be appearing on Th'
b'e View tomorrow morning to discuss Celebrity Appren'
b'tice and his new book Think Like A Champion!Donald '


Nous allons maintenant générer les couples (données, labels) à partir des séquences extraites :

In [60]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [61]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [62]:
dataset = sequences.map(split_input_target)

In [63]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'Be sure to tune in and watch Donald Trump on Late '
Target: b'e sure to tune in and watch Donald Trump on Late N'


Avant de pouvoir fournir les données au modèle, il est important de les ranger dans un ordre aléatoire et de les regrouper en batches.

Le paramètre `prefetch` permet d'organiser le chargement du prochain batch de données pendant que le modèle est en train de traiter le précédent.

In [64]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 50), dtype=tf.int64, name=None), TensorSpec(shape=(64, 50), dtype=tf.int64, name=None))>

# Construction du modèle

Le modèle sera composé de 3 couches seulement :

* `tf.keras.layers.Embedding`: La couche d'entrée, qui permet d'apprendre un descripteur de dimension`embedding_dim` à associer à chacun des caractères passés en entrée;
* `tf.keras.layers.GRU`: Une cellule récurrente, avec `rnn_units` neurones (que l'on pourrait tout à fait remplacer par un LSTM)
* `tf.keras.layers.Dense`: La couche de sortie, avec `vocab_size` neurones. Notez qu'on ne spécifie pas la fonction d'activation (`softmax`) car elle est intégrée directement dans la fonction de coût.

Pour chaque caractère de la séquence, le modèle produit le descripteur associé, applique un pas de temps du GRU et enfin applique la couche dense pour obtenir la prédiction du réseau :

![A drawing of the data passing through the model](https://drive.google.com/uc?id=1GYD8U9aF-MTC1XpJ3VKpY1b0clJuO4wb)

In [132]:
# Taille du vocabulaire
vocab_size = len(ids_from_chars.get_vocabulary())

# Dimension des descripteurs de caractères
embedding_dim = 256

# Nombre de neurones du GRU
rnn_units = 512

**Le bloc ci-dessous est à compléter** :

In [133]:
class MyModel(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim, rnn_units):
    super().__init__(self)
    # Définition de la couche d'Embedding
    self.embedding = tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim)# A COMPLETER
    self.gru = tf.keras.layers.GRU(rnn_units, # A COMPLETER
                                   return_sequences=True,
                                   return_state=True)
    self.dense1 = tf.keras.layers.Dense(vocab_size*2, activation="relu")
    self.dense2 = tf.keras.layers.Dense(vocab_size)#  A COMPLETER

  def call(self, inputs, states=None, return_state=False, training=False):
    x = inputs
    x = self.embedding(x, training=training)
    if states is None:
      states = self.gru.get_initial_state(x)
    x, states = self.gru(x, initial_state=states, training=training)
    x = self.dense1(x, training=training)
    x = self.dense2(x, training=training)

    if return_state:
      return x, states
    else:
      return x

N.B. Cette manière inhabituelle de définir le modèle (qui ressemble d'ailleurs beaucoup au formalisme Pytorch) est utile pour l'inférence. Nous aurions pu utiliser un modèle classique (construit avec `keras.Sequential`) mais cela ne nous aurait pas donné d'accès simple aux états internes du GRU. Il sera important de pouvoir manipuler cet état lorsque nous enchaînerons plusieurs prédictions successives, qui nécessiteront chaque fois de repartir de l'état obtenu lors de la prédiction précédente. Cette opération n'est pas possible avec le modèle Sequentiel que nous utilisons d'habitude.

In [134]:
model = MyModel(
    vocab_size=vocab_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)

Nous pouvons tester notre modèle sur le premier exemple d'apprentissage, pour vérifier les dimensions :

In [135]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 50, 260) # (batch_size, sequence_length, vocab_size)


Rq : +1 vocab lié à ids_from_chars (caractère->codage vecteur) qui prévoit d'ajouter un caractère inconnu au vocabulaire et donc c'est l'espace prévu pour ce caractère.

In [136]:
model.summary()

Model: "my_model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_9 (Embedding)     multiple                  66560     
                                                                 
 gru_9 (GRU)                 multiple                  1182720   
                                                                 
 dense_12 (Dense)            multiple                  266760    
                                                                 
 dense_13 (Dense)            multiple                  135460    
                                                                 
Total params: 1651500 (6.30 MB)
Trainable params: 1651500 (6.30 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# Entraînement du modèle

Le problème que nous cherchons à résoudre est celui d'une classification à `vocab_size` classes.

On utilise la fonction de coût `tf.keras.losses.sparse_categorical_crossentropy` car nos labels sont sous forme d'indices (et pas de *one-hot vectors*). Le flag `from_logits` positionné à `True` indique qu'il faut au préalable appliquer la fonction softmax à la sortie du réseau.

In [137]:
model.compile(optimizer='adam', loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True))

In [138]:
history = model.fit(dataset, epochs=10)

Epoch 1/10
1746/1746 [==============================] - 48s 23ms/step - loss: 1.9115
Epoch 2/10
1746/1746 [==============================] - 36s 20ms/step - loss: 1.4397
Epoch 3/10
1746/1746 [==============================] - 37s 20ms/step - loss: 1.3510
Epoch 4/10
1746/1746 [==============================] - 35s 19ms/step - loss: 1.3055
Epoch 5/10
1746/1746 [==============================] - 37s 20ms/step - loss: 1.2749
Epoch 6/10
1746/1746 [==============================] - 34s 19ms/step - loss: 1.2527
Epoch 7/10
1746/1746 [==============================] - 32s 18ms/step - loss: 1.2350
Epoch 8/10
1746/1746 [==============================] - 32s 18ms/step - loss: 1.2210
Epoch 9/10
1746/1746 [==============================] - 31s 17ms/step - loss: 1.2095
Epoch 10/10
1746/1746 [==============================] - 33s 17ms/step - loss: 1.2003


# Génération de texte

Pour générer un texte, il suffit de partir d'une séquence initiale, d'effectuer une prédiction, et de conserver l'état interne du modèle pour pouvoir le restaurer lors de la prochaine inférence, qui prendra en entrée la séquence initiale augmentée du caractère prédit précédemment.


La classe suivante permet de réaliser une prédiction :

In [139]:
class OneStep(tf.keras.Model):
  def __init__(self, model, chars_from_ids, ids_from_chars, temperature=0.1):
    super().__init__()
    self.temperature = temperature
    self.model = model
    self.chars_from_ids = chars_from_ids
    self.ids_from_chars = ids_from_chars


  @tf.function
  def generate_one_step(self, inputs, states=None):
    # Conversion des chaines de caractères en token IDs.
    input_chars = tf.strings.unicode_split(inputs, 'UTF-8')
    input_ids = self.ids_from_chars(input_chars).to_tensor()

    # Inférence du modèle
    # predicted_logits est de dimension [batch, char, next_char_logits]
    predicted_logits, states = self.model(inputs=input_ids, states=states,
                                          return_state=True)

    # Utilisation de la dernière prédiction seulement
    predicted_logits = predicted_logits[:, -1, :]
    predicted_logits = predicted_logits/self.temperature

    # Echantillonnage de la distribution de proba obtenue pour obtenir le prochain
    # caractère
    predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
    predicted_ids = tf.squeeze(predicted_ids, axis=-1) # removes dimensions of size 1 from the shape of a tensor

    # Conversion du token ID en caractère
    predicted_chars = self.chars_from_ids(predicted_ids)

    return predicted_chars, states

In [140]:
one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Il reste à appeler cette fonction dans une boucle pour générer un texte complet :

In [142]:
start = time.time()
states = None
next_char = tf.constant(['climate '])
result = [next_char]

for n in range(100):
  next_char, states = one_step_model.generate_one_step(next_char, states=states)
  result.append(next_char)

result = tf.strings.join(result)
end = time.time()
print(result[0].numpy().decode('utf-8'), '\n\n' + '_'*80)
print('\nRun time:', end - start)

climate the United States and the Fake News Media in the history of our Country that they are doing and get  

________________________________________________________________________________

Run time: 0.2644345760345459


A vous de jouer pour améliorer les résultats. Vous pouvez par exemple :      
- Jouer avec le paramètre de température dans la classe `OneStep` pour accentuer ou diminuer le caractère aléatoire des prédictions.
- Modifier le réseau en rajoutant des couches supplémentaires, ou en modifiant le nombre de neurones de la couche GRU.
- Modifier la préparation des données pour éviter le problème des séquences chevauchant plusieurs tweets
- Entraîner le modèle plus longtemps devrait également aider !

- Temperature de base = 1, si <1 fonctionne mieux mais si >1 fonctionne moins bien.
- on peut ajouter une séquence dense par exemple, mais dans ce cas, on doit mettre une fonction d'activation en sortie de la première couche, sinon on utilise une combinaison linéaire des entrées de dense1 = sorties de dense1 en entrée de dense2 ! Comme si on faisait rien...
- ici on a concaténé tous les tweets, c'est mieux de les traiter un par un. Et pour chaque tweets, d'extraire 1, ou 2 séquences de 50 caractères, ou bien de toutes les extraire (avancer de un aà chaque fois). Si séquence de moins de 50 caractères, on ajoute un masque au début avec du zéro padding (000000tweet).
- epochs de base = 5, mais avec 10 c'est mieux.